# Policy_Evaluation

Policy Evaluation：对于给定的策略$\pi$，计算它的value function，方法是通过iteration of Bellman Expectation backup。迭代公式如下所示：

$$
v_{k+1}(s)=\sum_{a \in A} \pi(a | s)\left(R_{s}^{a}+\gamma \sum_{s^{\prime} \in S} P_{s s^{\prime}}^{a} v_{k}\left(s^{\prime}\right)\right)\\
v^{k+1}=R^{\pi}+\gamma P^{\pi}v^k
$$


In [1]:
from IPython.core.debugger import set_trace
import numpy as np
import pprint
import sys
if "../" not in sys.path:
    sys.path.append("../") 
from lib.envs.gridworld import GridworldEnv

In [3]:
pp = pprint.PrettyPrinter(indent=2)
env = GridworldEnv()

In [26]:
def policy_eval(policy, env, discount_factor=1.0, theta=0.00001):
    """
    Evaluate a policy given an environment and a full description of the environment's dynamics.
    
    Args:
        policy: [S, A] shaped matrix representing the policy.
        env: OpenAI env. env.P represents the transition probabilities of the environment.
            env.P[s][a] is a list of transition tuples (prob, next_state, reward, done).
            env.nS is a number of states in the environment. 
            env.nA is a number of actions in the environment.
        theta: We stop evaluation once our value function change is less than theta for all states.
        discount_factor: Gamma discount factor.
    
    Returns:
        Vector of length env.nS representing the value function.
    """
    # Start with a random (all 0) value function
    V = np.zeros(env.nS) 
    V_tmp=np.zeros(env.nS)#修改1
    while True:
        delta = 0
        # For each state, perform a "full backup"
        for s in range(env.nS):
            v = 0
            # Look at the possible next actions
            for a, action_prob in enumerate(policy[s]):
                # For each action, look at the possible next states...
                for  prob, next_state, reward, done in env.P[s][a]:
                    # Calculate the expected value. Ref: Sutton book eq. 4.6.
                    v += action_prob * prob * (reward + discount_factor * V[next_state])
            # How much our value function changed (across any states)
            delta = max(delta, np.abs(v - V[s]))
            #V[s]=v
            V_tmp[s] = v #修改2
        V=V_tmp #修改3
        # Stop evaluating once our value function change is below a threshold
        if delta < theta:
            break
    return np.array(V)

In [27]:
random_policy = np.ones([env.nS, env.nA]) / env.nA
v = policy_eval(random_policy, env)

In [28]:
print("Value Function:")
print(v)
print("")

print("Reshaped Grid Value Function:")
print(v.reshape(env.shape))
print("")

Value Function:
[  0.         -13.99993311 -19.99990384 -21.99989416 -13.99993311
 -17.99991792 -19.99991088 -19.9999119  -19.99990384 -19.99991088
 -17.9999248  -13.99994386 -21.99989416 -19.9999119  -13.99994386
   0.        ]

Reshaped Grid Value Function:
[[  0.         -13.99993311 -19.99990384 -21.99989416]
 [-13.99993311 -17.99991792 -19.99991088 -19.9999119 ]
 [-19.99990384 -19.99991088 -17.9999248  -13.99994386]
 [-21.99989416 -19.9999119  -13.99994386   0.        ]]

